In [ ]:
# Import required packages
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model, metrics
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import pandas as pd
import math
import re

: 

In [ ]:
# Import dataset
# THIS NEEDS TO BE CHANGED BASED ON DEVICE
fin_df = pd.read_csv('/home/mia/Documents/College/CS4811/TwitterNLP/finance_data_II.csv')
# display(fin_df.head(3))
TOTAL_ROWS = len(fin_df.index) - 1
TRAIN_SIZE = math.ceil(TOTAL_ROWS * .75)

# Scale everything from 0 to 1
date = fin_df.pop('Date')           # Store the date column
col_names = list(fin_df.columns)    # Store the column names lost in conversion to numpy array

x = fin_df.values #returns a numpy array
min_max_scaler = MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
fin_df = pd.DataFrame(x_scaled, columns=col_names)
fin_df = pd.concat((date, fin_df), 1)

In [ ]:
# Import Twitter data
STOCK = 'SPX'
twt_df = pd.read_csv('/home/mia/Documents/College/CS4811/TwitterNLP/twt_data/SPX.csv')
# Round to the date
# round_time_pattern = " " + ".*"
# twt_df['Date'] = pd.to_datetime(twt_df['Date'])
# twt_df['Date'] = twt_df['Date'].dt.round('D')
# twt_df['Date'] = twt_df['Date'].astype(str)
count = 0
# for date in twt_df['Date']:
#     date = re.sub(round_time_pattern, '', date)
#     date = re.sub(r"\-",'/', date)
#     twt_df.loc[count, 'Date'] = date
#     count = count + 1

display(twt_df)
# display(fin_df)


In [ ]:
'''
combine the stock data and the twitter sentiment data using this function
Order: stock, twitter data, finance data
'''
def combine_data(STOCK, t_df, f_df):
    combined_df = pd.DataFrame()
    fin_name = "fin_" + STOCK
    twt_name = "twt_" + STOCK
    
    
    # Select date and stock for fin data
    f_df = f_df[['Date', STOCK]]
    f_df.rename(columns={STOCK: fin_name}, inplace=True)
    # display(f_df.head(2))
    # display(t_df.head(2))
    
    # Remove all dates from fin_df that don't occur in twt_df
    count = 0
    fin_dates = f_df.iloc[:,0]
    for date in fin_dates:
        # print(date)
        # print(twt_df['Date'].values)
        if date not in t_df['Date'].values:
            f_df.drop(fin_df[fin_df['Date'] == date].index, inplace = True)
            # print("no match")
           
            # fin_df.drop([date])

    # Rename the column for twt data
    t_df.rename(columns={"$" + STOCK: twt_name}, inplace=True)    

    # Remove move all dates in twt_df that don't occur in fin_df
    twt_dates = t_df.iloc[:,0]
    for date in twt_dates:
        if date  not in f_df['Date'].values:
            t_df.drop(t_df[t_df['Date'] == date].index, inplace = True)
              # print("no match")

    display(t_df)
    display(f_df)

    # THERE SHOULD BE A LINE FOR CONCATENTATING THE t_df and f_df int combined_df. IDK WHERE IT WENT
    # Here's how: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html

    return combined_df

In [ ]:
# Use the function to combine the twitter data and the stock data
combined_data_df = combine_data(STOCK, twt_df, fin_df)
display(combine_data_df)

In [ ]:
# Import dataset
# df = pd.read_csv('/home/mia/Documents/College/CS4811/TwitterNLP/finance_data.csv').convert_dtypes()
# TOTAL_ROWS = len(df.index) - 1
# TRAIN_SIZE = math.ceil(TOTAL_ROWS * .75)

# counter = 0
# for label, col in df.iteritems():
#     if label != 'Date':
#         df.loc[:, label] = col.str.replace("%", "").astype(float)

# # Scale everything from 0 to 1
# # Store the date column
# dates = df['Date']
# df = df.drop('Date', axis=1)
# df = (df-df.min())/(df.max()-df.min())
# df = pd.concat((dates, df), 1)

# display(df.head(3))

In [ ]:
# Add all training data to a DataFrame
df_train = pd.DataFrame()
df_train = df.iloc[0:TRAIN_SIZE]

# Add all the testing data to a DataFrame
df_test = pd.DataFrame()
df_test = df.iloc[TRAIN_SIZE:] 
display(df_train.head(5))


In [ ]:
# Left over from the regression
y_train = df_train['AMZN']
x_train = df_train.loc[:,'SPX':'JNJ']

x_train_lm = sm.add_constant(x_train)
lr_1 = sm.OLS(y_train, x_train_lm)
lr_1 = sm.OLS(y_train, x_train_lm).fit()
lr_1.summary()